In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = ( 
 SparkSession
 .builder
    .master("local[*]")
 .appName('spark_sql')
 .getOrCreate()
)

In [7]:

df = (
    spark
    .read
    .option('delimiter', ';')
    .option('header', 'true')
    .option('inferSchema', 'true')
    .option('enconding', 'ISO-8859-1')
     .csv('./dados/precos-gasolina-etanol-09.csv')
)
df.printSchema()

root
 |-- Regiao - Sigla: string (nullable = true)
 |-- Estado - Sigla: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Revenda: string (nullable = true)
 |-- CNPJ da Revenda: string (nullable = true)
 |-- Nome da Rua: string (nullable = true)
 |-- Numero Rua: string (nullable = true)
 |-- Complemento: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Data da Coleta: string (nullable = true)
 |-- Valor de Venda: string (nullable = true)
 |-- Valor de Compra: string (nullable = true)
 |-- Unidade de Medida: string (nullable = true)
 |-- Bandeira: string (nullable = true)



In [9]:
df.createOrReplaceTempView('combustiveis')


In [10]:
spark.sql("""
    select * from combustiveis where `Valor de Compra` IS NOT NULL
""").show()

+--------------+--------------+---------+-------+---------------+-----------+----------+-----------+------+---+-------+--------------+--------------+---------------+-----------------+--------+
|Regiao - Sigla|Estado - Sigla|Municipio|Revenda|CNPJ da Revenda|Nome da Rua|Numero Rua|Complemento|Bairro|Cep|Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|Bandeira|
+--------------+--------------+---------+-------+---------------+-----------+----------+-----------+------+---+-------+--------------+--------------+---------------+-----------------+--------+
+--------------+--------------+---------+-------+---------------+-----------+----------+-----------+------+---+-------+--------------+--------------+---------------+-----------------+--------+



In [11]:
view_precos = spark.sql("""
    select 
        `Estado - Sigla`, 
        `Produto`, 
        regexp_replace(`Valor de Venda`, ",", ".") as `Valor de Venda`,
        `Unidade de Medida`
    from combustiveis
""")

In [12]:
view_precos.createOrReplaceTempView('view_precos')
view_precos.show(5)

+--------------+------------------+--------------+-----------------+
|Estado - Sigla|           Produto|Valor de Venda|Unidade de Medida|
+--------------+------------------+--------------+-----------------+
|            SP|          GASOLINA|          5.39|       R$ / litro|
|            SP|            ETANOL|          3.49|       R$ / litro|
|            AC|          GASOLINA|          7.17|       R$ / litro|
|            AC|GASOLINA ADITIVADA|          7.22|       R$ / litro|
|            AC|            ETANOL|          4.96|       R$ / litro|
+--------------+------------------+--------------+-----------------+
only showing top 5 rows



In [13]:
df_diferenca_precos = spark.sql("""
    select 
        `Estado - Sigla`,
        `Produto`,
        `Unidade de Medida`,
        MIN(`Valor de Venda`) as menor_valor,
        MAX(`Valor de Venda`) as maior_valor,
        MAX(`Valor de Venda`) - MIN(`Valor de Venda`) as diferenca
   from view_precos
   group by all
   order by diferenca DESC
""")
df_diferenca_precos.show(5)

+--------------+------------------+-----------------+-----------+-----------+------------------+
|Estado - Sigla|           Produto|Unidade de Medida|menor_valor|maior_valor|         diferenca|
+--------------+------------------+-----------------+-----------+-----------+------------------+
|            RS|GASOLINA ADITIVADA|       R$ / litro|       5.39|       9.79|4.3999999999999995|
|            SP|GASOLINA ADITIVADA|       R$ / litro|       5.29|       8.89|3.6000000000000005|
|            SP|          GASOLINA|       R$ / litro|       4.99|       7.99|               3.0|
|            SP|            ETANOL|       R$ / litro|       3.19|       5.99|2.8000000000000003|
|            RJ|GASOLINA ADITIVADA|       R$ / litro|       5.49|       7.99|               2.5|
+--------------+------------------+-----------------+-----------+-----------+------------------+
only showing top 5 rows

